# Import

In [ ]:
#!/opt/conda/bin/python3

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.tensorboard import SummaryWriter  # TensorBoard import

# Define the LSTM model for actuator identification

In [ ]:
class ActuatorLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        """
        Initialize the LSTM model.
        :param input_size: Number of input features (joint position, velocity, time)
        :param hidden_size: Number of units in the LSTM hidden layer
        :param num_layers: Number of LSTM layers
        :param output_size: Number of output features (torque, temperature)
        """
        super(ActuatorLSTM, self).__init__()
        
        # Define the LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        
        # Define a fully connected (dense) layer to map LSTM outputs to desired outputs
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        """
        Forward pass through the network.
        :param x: Input sequence with shape (batch_size, seq_len, input_size)
        :return: Predicted torque and temperature with shape (batch_size, seq_len, output_size)
        """

        h0 = torch.zeros(self.lstm.num_layers, x.size(0), self.lstm.hidden_size).to(x.device)
        c0 = torch.zeros(self.lstm.num_layers, x.size(0), self.lstm.hidden_size).to(x.device)
        
        out, _ = self.lstm(x, (h0, c0))  # LSTM output
        out = self.fc(out[:, -1, :])     # Fully connected layer (last timestep)
        return out

# Hyperparameters

In [ ]:
input_size = 3          # e.g., position, velocity, time
hidden_size = 50
output_size = 2         # e.g., torque and temperature
num_layers = 2
learning_rate = 0.001
num_epochs = 100
batch_size = 32

# Initialize model, loss, optimizer, and TensorBoard writer

In [ ]:
model = ActuatorLSTM(input_size, hidden_size, output_size, num_layers).to('cuda')
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
writer = SummaryWriter(log_dir='./runs/actuator_lstm')  # TensorBoard writer

### Example input data (batch_size=16, sequence_length=10, input_size=3). Here, batch_size is the number of samples in a batch seq_len is the length of the time sequence fed to the LSTM. Sample dataset (replace x_samples and y_samples with your actual data)

In [ ]:
x_samples = torch.rand((1000, 10, input_size))  # e.g., 1000 sequences of length 10
y_samples = torch.rand((1000, output_size))     # Target outputs
dataset = TensorDataset(x_samples, y_samples)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Forward pass

In [ ]:
#output = model(x_sample)
#print("Output shape:", output.shape)  # Should be (batch_size, sequence_length, output_size)

# Training loop

In [ ]:
# Training loop
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for i, (x_batch, y_batch) in enumerate(dataloader):
        x_batch, y_batch = x_batch.to('cuda'), y_batch.to('cuda')

        # Forward pass
        outputs = model(x_batch)
        loss = criterion(outputs, y_batch)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
    
    # Calculate average loss and log it to TensorBoard
    avg_loss = epoch_loss / len(dataloader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')
    writer.add_scalar('Loss/train', avg_loss, epoch)  # Log to TensorBoard
    
    # Save model checkpoints every 10 epochs
    if (epoch + 1) % 10 == 0:
        torch.save(model.state_dict(), f'./checkpoints/actuator_lstm_epoch_{epoch+1}.pth')

# Close the TensorBoard writer
writer.close()
print("Training complete.")